# 1.7 项目

一个计算英国,法国,日本,澳大利亚当前时间的app,项目将用到flask和datetime这三个库,我们还是用之前的模板,只是修改app文件和index

In [44]:
%%writefile ../codes/worldtime/worldtime.py
from flask import Flask,render_template,url_for,flash,redirect
from flask.ext.script import Manager
from flask.ext.bootstrap import Bootstrap


from flask.ext.wtf import Form
from wtforms import RadioField, SubmitField 
from wtforms.validators import Required
  
import time
from datetime import timedelta, timezone,datetime

app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'# 新增
app.debug = True
bootstrap = Bootstrap(app)
manager = Manager(app)

#新增
class TimeForm(Form):
    country = RadioField("Which country? ", 
                      choices=[('Japan', 'Japan'), ('UK', 'UK'), ('France', 'France'),("Australia","Australia")],
                      validators=[Required()])
    submit = SubmitField('Submit')

@app.template_filter('changeTime')
def changeTime(text):
    now_utc = datetime.utcnow().replace(tzinfo=timezone.utc)
    return str(now_utc.astimezone(timezone(timedelta(hours=text))).strftime('%a, %b %d %H:%M'))    


@app.route('/',methods = ["POST","GET"])
def worldtime():
    timezones = {
        "China":(8,u"北京"),
        "Japan":(9,u"东京"),
        "UK":(0,u"伦敦"),
        "France":(2,u"巴黎"),
        "Australia":(10,u"悉尼")
    }
    chinatimezone,chinacity = timezones.get("China")
    timezone = chinatimezone
    city = chinacity
    timeform = TimeForm()
    if timeform.validate_on_submit():
        country = timeform.country.data
        if country:
            country = str(timeform.country.data)
            timezone,city = timezones.get(country)
        else:
            flash("Looks like there is something wrong!")
            return redirect(url_for('worldtime'))   
    
        
    return render_template('myapp/index.html',timeform=timeform,
                           chinacity = chinacity,
                           chinatime=chinatimezone,
                           city= city, 
                           timezone=timezone)


Overwriting ../codes/worldtime/worldtime.py


In [45]:
%%writefile ../codes/worldtime/templates/myapp/index.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}hello world!{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <h1>World's Time Welcome!</h1>
            <p>{{chinacity}}时间: {{chinatime|changeTime}}</p>
            {% if city != chinacity%}
                <p>{{city}}时间:{{ timezone|changeTime }}</p>
            {% endif%}
            {{ wtf.quick_form( timeform,form_type='horizontal') }}
            
        </div>      
    </div>    
    
{{ super() }}
{% endblock %}

Overwriting ../codes/worldtime/templates/myapp/index.html


In [46]:
%%writefile ../codes/worldtime/manager.py
#coding:UTF-8
import os
import sys

root = os.path.dirname(__file__)

sys.path.insert(0, os.path.join(root, 'site-packages'))

from worldtime import manager

if __name__ == '__main__':
    manager.run()

Overwriting ../codes/worldtime/manager.py


In [47]:
!python3 ../codes/worldtime/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [20/Jan/2016 15:57:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:57:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Jan/2016 15:57:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:57:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:57:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:57:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:57:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:58:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 15:58:05] "POST / HTTP/1.1" 200 -
^C


# 总结

用到的模块:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展
flask-wtf|构建表单